ran within geoai-immap/notebooks/

# init

In [8]:
import pandas as pd
import numpy as np
from scipy.integrate import trapz as integrate
import itertools
# source: https://en.wikipedia.org/wiki/File:Composite_trapezoidal_rule_illustration.png

import sys
sys.path.insert(0, '../utils')
from model_utils import AREA_CODES, calculate_precision_recall

In [9]:
# # get results data to vm
# !gsutil cp -r gs://immap-results/* results/
# !gsutil cp -r gs://immap-training/20200509_dataset.csv ../data/20200509_dataset.csv

In [10]:
%%time
# grid-area mapping
df = pd.read_csv('../data/20200509_dataset.csv')
grid_area = (df[['uid', 'area']]
             .drop_duplicates()
             .sort_values('uid')
             .rename(columns = {'uid': 'grid_id'}))
assert grid_area[['grid_id']].drop_duplicates().shape[0] == grid_area.shape[0]

# list of model strings per model type
models = {
    'logistic_regression': [
        'penaltyl1_c0.001',
        'penaltyl1_c0.010',
        'penaltyl1_c0.100',
        'penaltyl1_c1.000',
        'penaltyl2_c0.001',
        'penaltyl2_c0.010',
        'penaltyl2_c0.100',
        'penaltyl2_c1.000',
    ], 
    'random_forest': [
        'nestimators100_maxdepth5_minsamplessplit5_minsamplesleaf5',
        'nestimators100_maxdepth8_minsamplessplit15_minsamplesleaf1',
        'nestimators300_maxdepth8_minsamplessplit5_minsamplesleaf5',
        'nestimators800_maxdepth12_minsamplessplit15_minsamplesleaf2',
        'nestimators800_maxdepth8_minsamplessplit2_minsamplesleaf10',
    ],
    'linear_svc': [
        'c0.001',
        'c0.010',
        'c0.100',
        'c1.000',
    ],
}

neg_samplings = ['10k_results','30k_results','50k_results']

model_types = list(models.keys())
areas = list(AREA_CODES.values())

CPU times: user 7.87 s, sys: 2.05 s, total: 9.91 s
Wall time: 9.91 s


# functions

In [11]:
def get_data(neg_sample, model_type, model):
    filedir = '../results/' + neg_sample + '/' + model_type + '_grid_preds_' + model + '.csv'
    return pd.read_csv(filedir)

In [12]:
def get_best_model(muncity, model_type, neg_sample):
    "compares models using area under the curve"
    models_ = models[model_type]
    sorted_aucs = []
    for model in models_:
        df = get_data(neg_sample, model_type, model)
        df2 = pd.merge(df, grid_area, how = 'left')
        area_code = [k for k,v in AREA_CODES.items() if v == muncity][0]
        df3 = df2.query("area == " + str(area_code))
        series_ = calculate_precision_recall(df3)['recall'] # <-------specifies which curve to optimize
        sorted_aucs.append(integrate(series_))
    assert len(df) == len(df2)
    assert len(df3) < len(df)
    best_model = [models_[i] for i in range(len(models_)) if sorted_aucs[i] == max(sorted_aucs)][0]
    return best_model

In [13]:
def intersect_membership(model1, model2, top = 0.1):
    "accepts 2 models, each defined as a tuple of 3 items: neg_sample, model_type, best_model"
    neg_sample1, model_type1, model1_ = model1
    neg_sample2, model_type2, model2_ = model2
    df1 = get_data(neg_sample1, model_type1, model1_)
    df2 = get_data(neg_sample2, model_type2, model2_)
    df1 = df1.sort_values('y_pred', ascending=False)
    df2 = df2.sort_values('y_pred', ascending=False)
    
    # get top10%
    df1_ = df1[0:int(df1.shape[0]*(0.1))]
    df2_ = df2[0:int(df2.shape[0]*(0.1))]

    # convert to set
    set1 = set(list(df1_['grid_id']))
    set2 = set(list(df2_['grid_id']))

    return len(set1.intersection(set2)) / len(set1.union(set2)) #max(len(set1), len(set2))

In [14]:
def calc_matrix(muncity):
    """
    Produce 9x9 matrix per municipality describing top10% membership intersections between models
    """
    best_models = []
    for neg_sample in neg_samplings:
        for model_type in model_types:
            models_ = models[model_type]
            best_model = get_best_model(muncity, model_type, neg_sample)
            best_models.append((neg_sample, model_type, best_model))
    assert len(best_models) == 9
    cart_prod = list(itertools.product(best_models, best_models))
    assert len(cart_prod) == 81
    pairs = []
    for i in range(len(cart_prod)):
        model1, model2 = cart_prod[i]
        mem_int = intersect_membership(model1, model2, top = 0.1)
        pairs.append((str(model1), str(model2), mem_int))
    
    return pairs

# main

In [15]:
areas

['Maicao',
 'Riohacha',
 'Uribia',
 'Arauca',
 'Cucuta',
 'Tibu',
 'Arauquita',
 'Soacha',
 'Bogota']

In [37]:
def get_best_model_v2(muncity):
    "compares models using area under the curve"
    models_2 = []
    sorted_aucs = []
    for neg_sample in neg_samplings:
        for model_type in model_types:
            models_ = models[model_type]
            for model in models_:
                df = get_data(neg_sample, model_type, model)
                df2 = pd.merge(df, grid_area, how = 'left')
                area_code = [k for k,v in AREA_CODES.items() if v == muncity][0]
                df3 = df2.query("area == " + str(area_code))
                series_ = calculate_precision_recall(df3)['recall'] # <-------specifies which curve to optimize
                models_2.append(model_type + '_' + model)
                sorted_aucs.append(integrate(series_))
    assert len(df) == len(df2)
    assert len(df3) < len(df)
    best_model, sorted_auc = [(models_2[i], sorted_aucs[i]) for i in range(len(models_2)) if sorted_aucs[i] == max(sorted_aucs)][0]
    return best_model, sorted_auc

In [33]:
neg_sample = neg_samplings[0]
model_type = model_types[1]
model = models[model_type][1]
muncity = 'Bogota'#'Arauquita'

df = get_data(neg_sample, model_type, model)

In [35]:
series_ = calculate_precision_recall(df)['recall'] # <-------specifies which curve to optimize
integrate(series_)

90.375

In [38]:
get_best_model_v2('Bogota')

('logistic_regression_penaltyl1_c1.000', 92.5)

In [39]:
get_best_model_v2('Arauca')

('random_forest_nestimators800_maxdepth12_minsamplessplit15_minsamplesleaf2',
 87.64285714285714)

In [40]:
get_best_model_v2('Riohacha')

('logistic_regression_penaltyl1_c0.100', 94.5)

In [20]:
%%time
for area in areas:
#     area = 'Maicao'
    pairs = calc_matrix(area)
    df = pd.DataFrame(pairs)
    df.columns = ['model1', 'model2', 'intersection']
    # df.sort_values('intersection')
    # df.to_csv('pairs_' + area + '.csv')

    # # convert to matrix
    # df2 = pd.DataFrame(np.array(df['intersection']).reshape(9,9))
    # names = list(df['model2'][0:9])
    # df2.columns = names
    # df2[area] = names
    # df2.set_index(area, inplace = True)
    # df2.to_csv('matrix_' + area + '.csv')

    df['arch'] = ''
    for i in range(len(df)):
        row = df.iloc[i,:]
        arch1 = row.model1.split("', '")[1]
        arch2 = row.model2.split("', '")[1]
        if arch1 == arch2:
            df.loc[i,'arch'] = arch1
        else:
            df.loc[i,'arch'] = 'EXCLUDE'

    with pd.ExcelWriter('matrixv2_' + area + '.xlsx') as writer:
        for model_type in model_types:
            # model_type = 'logistic_regression'
            df_ = df.query("arch == '" + model_type + "'")
            df2 = pd.DataFrame(np.array(df_['intersection']).reshape(3,3))
            names = list(df_['model2'][0:3])
            df2.columns = names
            df2[area] = names
            df2.set_index(area, inplace = True)
            # df2.to_csv('matrix_' + area + '_' + model_type + '.csv')
            df2.to_excel(writer, sheet_name=area + '_' + model_type)

CPU times: user 16.2 s, sys: 88.5 ms, total: 16.3 s
Wall time: 16.2 s


# check

compare to 03 optimization notebooks

In [11]:
# for riohacha, 10K best logistic model should be L1_c0.1
# PASS
df2

,"('10k_results', 'logistic_regression', 'penaltyl1_c0.100')","('10k_results', 'random_forest', 'nestimators100_maxdepth8_minsamplessplit15_minsamplesleaf1')","('10k_results', 'linear_svc', 'c0.100')","('30k_results', 'logistic_regression', 'penaltyl1_c0.100')","('30k_results', 'random_forest', 'nestimators100_maxdepth8_minsamplessplit15_minsamplesleaf1')","('30k_results', 'linear_svc', 'c0.100')","('50k_results', 'logistic_regression', 'penaltyl1_c0.010')","('50k_results', 'random_forest', 'nestimators800_maxdepth12_minsamplessplit15_minsamplesleaf2')","('50k_results', 'linear_svc', 'c0.010')"
Riohacha,,,,,,,,,
"('10k_results', 'logistic_regression', 'penaltyl1_c0.100')",1.000000,0.409836,0.869565,0.869565,0.482759,0.755102,0.720000,0.433333,0.720000
"('10k_results', 'random_forest', 'nestimators100_maxdepth8_minsamplessplit15_minsamplesleaf1')",0.409836,1.000000,0.433333,0.482759,0.869565,0.508772,0.508772,0.829787,0.535714
"('10k_results', 'linear_svc', 'c0.100')",0.869565,0.433333,1.000000,0.829787,0.508772,0.791667,0.720000,0.457627,0.720000
"('30k_results', 'logistic_regression', 'penaltyl1_c0.100')",0.869565,0.482759,0.829787,1.000000,0.508772,0.755102,0.720000,0.457627,0.720000
"('30k_results', 'random_forest', 'nestimators100_maxdepth8_minsamplessplit15_minsamplesleaf1')",0.482759,0.869565,0.508772,0.508772,1.000000,0.563636,0.563636,0.791667,0.592593
"('30k_results', 'linear_svc', 'c0.100')",0.755102,0.508772,0.791667,0.755102,0.563636,1.000000,0.791667,0.482759,0.791667
"('50k_results', 'logistic_regression', 'penaltyl1_c0.010')",0.720000,0.508772,0.720000,0.720000,0.563636,0.791667,1.000000,0.482759,0.829787
"('50k_results', 'random_forest', 'nestimators800_maxdepth12_minsamplessplit15_minsamplesleaf2')",0.433333,0.829787,0.457627,0.457627,0.791667,0.482759,0.482759,1.000000,0.535714
"('50k_results', 'linear_svc', 'c0.010')",0.720000,0.535714,0.720000,0.720000,0.592593,0.791667,0.829787,0.535714,1.000000


In [13]:
# for soacha, 30K best svc model should be c0.01
# FAIL
df2

,"('10k_results', 'logistic_regression', 'penaltyl1_c0.010')","('10k_results', 'random_forest', 'nestimators100_maxdepth8_minsamplessplit15_minsamplesleaf1')","('10k_results', 'linear_svc', 'c0.010')","('30k_results', 'logistic_regression', 'penaltyl1_c0.010')","('30k_results', 'random_forest', 'nestimators100_maxdepth8_minsamplessplit15_minsamplesleaf1')","('30k_results', 'linear_svc', 'c0.010')","('50k_results', 'logistic_regression', 'penaltyl1_c0.010')","('50k_results', 'random_forest', 'nestimators100_maxdepth8_minsamplessplit15_minsamplesleaf1')","('50k_results', 'linear_svc', 'c0.010')"
Soacha,,,,,,,,,
"('10k_results', 'logistic_regression', 'penaltyl1_c0.010')",1.000000,0.457627,0.791667,0.869565,0.508772,0.829787,0.869565,0.535714,0.829787
"('10k_results', 'random_forest', 'nestimators100_maxdepth8_minsamplessplit15_minsamplesleaf1')",0.457627,1.000000,0.535714,0.482759,0.869565,0.563636,0.508772,0.869565,0.535714
"('10k_results', 'linear_svc', 'c0.010')",0.791667,0.535714,1.000000,0.755102,0.592593,0.869565,0.791667,0.622642,0.791667
"('30k_results', 'logistic_regression', 'penaltyl1_c0.010')",0.869565,0.482759,0.755102,1.000000,0.535714,0.829787,0.954545,0.563636,0.829787
"('30k_results', 'random_forest', 'nestimators100_maxdepth8_minsamplessplit15_minsamplesleaf1')",0.508772,0.869565,0.592593,0.535714,1.000000,0.622642,0.563636,0.869565,0.592593
"('30k_results', 'linear_svc', 'c0.010')",0.829787,0.563636,0.869565,0.829787,0.622642,1.000000,0.869565,0.653846,0.911111
"('50k_results', 'logistic_regression', 'penaltyl1_c0.010')",0.869565,0.508772,0.791667,0.954545,0.563636,0.869565,1.000000,0.592593,0.829787
"('50k_results', 'random_forest', 'nestimators100_maxdepth8_minsamplessplit15_minsamplesleaf1')",0.535714,0.869565,0.622642,0.563636,0.869565,0.653846,0.592593,1.000000,0.622642
"('50k_results', 'linear_svc', 'c0.010')",0.829787,0.535714,0.791667,0.829787,0.592593,0.911111,0.829787,0.622642,1.000000


In [20]:
# displayed best, c0.01, is same as c1.00 as recorded in notebook
def get_models(muncity, model_type, neg_sample):
    "compares models using area under the curve"
    models_ = models[model_type]
    sorted_aucs = []
    for model in models_:
        df = get_data(neg_sample, model_type, model)
        df2 = pd.merge(df, grid_area, how = 'left')
        area_code = [k for k,v in AREA_CODES.items() if v == muncity][0]
        df3 = df2.query("area == " + str(area_code))
        series_ = calculate_precision_recall(df3)['recall'] # <-------specifies which curve to optimize
        sorted_aucs.append(integrate(series_))
    assert len(df) == len(df2)
    assert len(df3) < len(df)
    # best_model = [models_[i] for i in range(len(models_)) if sorted_aucs[i] == max(sorted_aucs)][0]
    return models_, sorted_aucs

get_models('Soacha', 'linear_svc', '30k_results')

(['c0.001', 'c0.010', 'c0.100', 'c1.000'], [84.5, 87.5, 87.5, 87.5])

In [ ]:
# # test 1
# neg_sample = neg_samplings[0]
# model_type = model_types[1]
# model = models[model_type][1]
# muncity = 'Maicao'#'Arauquita'

# df = get_data(neg_sample, model_type, model)
# df2 = pd.merge(df, grid_area, how = 'left')
# area_code = [k for k,v in AREA_CODES.items() if v == muncity][0]
# df3 = df2.query("area == " + str(area_code))

# df3.head(3).sort_values('y_pred', ascending = False)